In [248]:
import os
import torch
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from deepcase_copy.context_builder.loss import LabelSmoothing
from deepcase_copy.context_builder.context_builder import ContextBuilder
from deepcase_copy.interpreter.interpreter import Interpreter
from deepcase_copy.interpreter.utils import group_by, sp_unique
from sklearn.model_selection import train_test_split

def disable_dropout(m):
    if isinstance(m, torch.nn.Dropout):
        m.p = 0.0 

def to_cuda(item):
    if torch.cuda.is_available():
        return item.to('cuda')
    return item

def get_unique_indices_per_row(tensor):
    indices_list = []
    row_list = []
    indices_list_set = set()
    for row in range(len(tensor)):
        curr = tuple(tensor[row])
        if isinstance(tensor, torch.Tensor):
            curr = tuple(tensor[row].tolist())
        if curr in indices_list_set:
            continue
        row_list.append(curr)
        indices_list.append(row)
        indices_list_set.add(curr)
    return indices_list

def get_unique_indices_per_row_or_file(tensor, f_name):
    if os.path.exists(f_name):
        return torch.load(f_name)
    t = get_unique_indices_per_row(tensor)
    torch.save(t, f_name)
    return t  

ALPHA=0.01
EPSILON=0.5
MAX_ITER = 100
PREDICT_THRESHOLD = 0.2
builder = to_cuda(ContextBuilder.load('save/builder.save'))
builder.apply(disable_dropout)
interpreter = Interpreter.load('save/interpreter.save', builder)
criterion = LabelSmoothing(builder.decoder_event.out.out_features, 0.1)
attention_query = True

with open('save/sequences.save', 'rb') as infile:
    data = torch.load(infile)
    context = data["context"] # 172572
    events  = data["events"]
    labels  = data["labels"]
    
    indices = get_unique_indices_per_row_or_file(context, 'save/context.pt')    
    train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42, stratify=labels[indices])
    
    context_train = to_cuda(context[train_indices]) #
    context_test  = to_cuda(context[test_indices])  # 4392
    
    events_train  = to_cuda(events[train_indices])
    events_test   = to_cuda(events[test_indices])
    
    labels_train  = to_cuda(labels[train_indices])
    labels_test   = to_cuda(labels[test_indices])

def to_one_hot(t):
    return builder.embedding_one_hot(t)

def max_to_one(tensor):
    max_indices = torch.argmax(tensor, dim=-1, keepdim=True)
    result = torch.zeros_like(tensor)
    result.scatter_(-1, max_indices, 1.0)
    return result

def to_output(context_chosen):
    return builder.predict(to_cuda(to_one_hot(context_chosen)))

def get_perturbation_change(trace, original):
    a = original - EPSILON
    b = (trace >= a).float() * trace + (trace < a).float() * a
    c = (b > original + EPSILON).float() * (original + EPSILON) + (b <= original + EPSILON).float() * b
    return max_to_one(c)

def get_performance(output, num):
    results_picked = torch.topk(output[0][0][0], len(output[0][0][0]))
    exp = results_picked.values.exp()
    res_indices = results_picked.indices
    for j in range(len(output[0][0][0])):
        if res_indices[j].item() == num:
            return exp[j] < PREDICT_THRESHOLD
    return False

def get_changes_list(start, final):
    perturbations_made = []
    for i, (s, f) in enumerate(zip(start, final)):
        if s != f:
            perturbations_made.append((i, f.item()))
    return perturbations_made

def get_iter_count(context_given, target_given):
    for current_trace_num in range(len(context_given)):
        con, e = context_given[current_trace_num], target_given.unsqueeze(1)[current_trace_num]
        con.resize_(1, con.size()[-1])
        for i in range(MAX_ITER):
            new_context = bim_no_iter(con, e, i)
            if con[0].tolist() != new_context.tolist():
                return i
    return -1

def get_perturbations(context_chosen, event_chosen):
    perturbed_collected_main = []
    perturbed_indices_main = []
    states = [0, 0, 0]
    for current_trace_num in tqdm(range(len(context_chosen))):
        con, e = context_chosen[current_trace_num], event_chosen.unsqueeze(1)[current_trace_num]
        con.resize_(1, con.size()[-1])
        if get_performance(to_output(con), e[0]):
            states[0] += 1
        else:
            perturbed_result = bim_attack(con, e)
            if perturbed_result is not None:
                states[1] += 1
                perturbed_collected_main.append(perturbed_result[0])
                perturbed_indices_main.append(current_trace_num)
            else:
                states[2] += 1
    return to_cuda(torch.tensor(perturbed_collected_main)), to_cuda(torch.tensor(perturbed_indices_main)), to_cuda(torch.tensor(states))

def get_perturbations_or_file(context_chosen, event_chosen):
    f_name_perturbed = f"perturbed/collected/{ALPHA=}, {EPSILON=}.pt"
    f_name_indices = f"perturbed/indices/{ALPHA=}, {EPSILON=}.pt"
    f_name_distribution = f"perturbed/distribution/{ALPHA=}, {EPSILON=}.pt"
    if os.path.exists(f_name_perturbed) and os.path.exists(f_name_indices) and os.path.exists(f_name_distribution):
        print(f"Loading {f_name_perturbed}")
        print(f"Loading {f_name_indices}")
        print(f"Loading {f_name_distribution}")
        return to_cuda(torch.load(f_name_perturbed)), to_cuda(torch.load(f_name_indices)), to_cuda(torch.load(f_name_distribution))
    perturb_main, indices_main, result_main = get_perturbations(context_chosen, event_chosen)
    torch.save(perturb_main, f_name_perturbed)
    torch.save(indices_main, f_name_indices)
    torch.save(result_main, f_name_distribution)
    return perturb_main, indices_main, result_main

def get_possible_combinations(perturbations_made):
    subsets = []
    for r_index in range(1, len(perturbations_made)):
        subsets.extend(itertools.combinations(perturbations_made, r_index))
    result = [list(subset) for subset in subsets]
    return result

def get_minimum_change_for_perturbation(perturbed_chosen, context_chosen, events_chosen):
    for combination in get_possible_combinations(get_changes_list(context_chosen, perturbed_chosen)):
        copy = to_cuda(context_chosen.clone().detach())
        for index_of_change, value_of_change in combination:
            copy[index_of_change] = value_of_change
        copy.resize_(1, copy.size()[-1])
        if get_performance(to_output(copy), events_chosen):
            return copy.squeeze(0)
    return perturbed_chosen

def get_minimum_change_for_perturbation_attention_query(perturbed_chosen, context_chosen, events_chosen):
    trace_combinations = []
    for combination in get_possible_combinations(get_changes_list(context_chosen, perturbed_chosen)):
        copy = to_cuda(context_chosen.clone().detach())
        for index_of_change, value_of_change in combination:
            copy[index_of_change] = value_of_change
        trace_combinations.append(copy)
    if len(trace_combinations) == 0:
        return perturbed_chosen
    vectors, mask = interpreter.attended_context(
        X           = to_one_hot(torch.stack(trace_combinations)),
        y           = to_cuda(torch.full((len(trace_combinations),1), events_chosen.item())),
        iterations  = 100
    )
    mask_indices = torch.where(~mask)[0]
    chosen = perturbed_chosen
    if len(mask_indices) != 0:
        chosen = trace_combinations[mask_indices[0]]
    return chosen
    
def get_shortcuts(perturbed_chosen, indices_chosen, context_chosen, events_chosen):
    pick_list = []
    for perturbed_element  in tqdm(zip(perturbed_chosen, context_chosen[indices_chosen], events_chosen[indices_chosen]), total=len(perturbed_chosen)):
        pick_list.append(get_minimum_change_for_perturbation(*perturbed_element))
    return to_cuda(torch.stack(pick_list))

def get_shortcuts_or_file(perturbed_chosen, indices_chosen, context_chosen, events_chosen):
    f_name = f"perturbed/minimized/{ALPHA=}, {EPSILON=}.pt"
    if os.path.exists(f_name):
        print(f"Loading {f_name}")
        return torch.load(f_name)
    pick_list = get_shortcuts(perturbed_chosen, indices_chosen, context_chosen, events_chosen)
    torch.save(pick_list, f_name)
    return pick_list
    
def get_shortcuts_attention_query(perturbed_chosen, indices_chosen, context_chosen, events_chosen):
    pick_list = []
    for perturbed_element  in tqdm(zip(perturbed_chosen, context_chosen[indices_chosen], events_chosen[indices_chosen]), total=len(perturbed_chosen)):
        pick_list.append(get_minimum_change_for_perturbation_attention_query(*perturbed_element))
    return to_cuda(torch.stack(pick_list))

def get_shortcuts_attention_query_or_file(perturbed_chosen, indices_chosen, context_chosen, events_chosen):
    f_name = f"perturbed/minimized_attention_query/{ALPHA=}, {EPSILON=}.pt"
    if os.path.exists(f_name):
        print(f"Loading {f_name}")
        return torch.load(f_name)
    pick_list = get_shortcuts_attention_query(perturbed_chosen, indices_chosen, context_chosen, events_chosen)
    torch.save(pick_list, f_name)
    return pick_list
    
def get_dist(trace_dist):
    tensor_tuples = [tuple(t) if len(t) > 0 else (t.item(),) for t in trace_dist]
    index_groups = {}
    for idx, tensor_tuple in enumerate(tensor_tuples):
        if tensor_tuple not in index_groups:
            index_groups[tensor_tuple] = []
        index_groups[tensor_tuple].append(idx)
    sorted_groups = sorted(index_groups.items(), key=lambda x: (len(x[0]), x[0]))
    return [idx for _, indices_l in sorted_groups for idx in indices_l]
    
def bim_attack(context_chosen, event_chosen):
    change = 0
    original_context = to_cuda(to_one_hot(context_chosen).clone().detach())
    context_processed = to_cuda(to_one_hot(context_chosen).clone().detach())
    for iteration in range(MAX_ITER):
        context_processed.requires_grad_(True)
        output = builder.predict(context_processed)
        if get_performance(output, event_chosen[0]):
            return torch.argmax(context_processed, dim=-1).tolist()[0], iteration
        loss = criterion(output[0][0], event_chosen)
        context_processed.retain_grad()
        loss.backward(retain_graph=True)
        grad = context_processed.grad.sign()
        change += ALPHA * grad
        context_processed = get_perturbation_change(context_processed + change, original_context)
    return None
    
def format_results(results):
    results_picked = torch.topk(results[0][0][0], 3)
    exp = results_picked.values.exp()
    res_indices = results_picked.indices
    return ", ".join([f"{format_list([res_indices[j].item()])} {'{:.3f}'.format(exp[j])}" for j in range(3)])
  
def format_changes(start, final):
    changes = []
    same = []
    for s, f in zip(start, final):
        if s == f:
            changes.append("-")
            same.append(final)
        else:
            changes.append(f)
            same.append("XX")
    return format_list(changes), format_list(same)

def format_list(li):
    return f"[{", ".join([f'{num:2}' for num in li])}]"
            
def format_trace_prediction(current_trace_num, trace):
    start = " "*(len(str(current_trace_num)) + 2)
    trace = to_cuda(trace.clone().detach())
    trace.resize_(1, trace.size()[-1])
    return f"{start}{format_list(trace)} -> {format_results(to_output(trace))}\n"
            
def format_perturbation(perturb_chosen, perturb_int, context_chosen, event_chosen, current_trace_num):
    result_string = f"{current_trace_num}: {format_list(context_chosen[0].tolist())} == {event_chosen.tolist()} Changed [{perturb_int}], Perturbations [{len(get_changes_list(context_chosen, perturb_chosen))}]\n"
    result_string += format_trace_prediction(current_trace_num, context_chosen)    
    result_string += format_trace_prediction(current_trace_num, perturb_chosen)    
    changed_entries, same_entries = format_changes(context_chosen, perturb_chosen)
    result_string += f"{" "*(len(str(current_trace_num)) - 1)}== {same_entries}\n{" "*(len(str(current_trace_num)) - 1)}-> {changed_entries}\n\n"
    return result_string

def format_attention(x_mask, context_mask, vectors, neighbours, distance, scores):
    data_collected = [] 
    for i in range(len(context_mask)):
        l_value = torch.tensor(vectors[context_mask][i].toarray()[0])
        l_indices = torch.nonzero(l_value, as_tuple=False).squeeze(1)
        data_collected.append({
            "trace": x_mask[context_mask][i],
            "indices": l_indices.tolist(),
            "value": l_value[l_indices].tolist(),
            "neighbour": f"{{{neighbours[i]:5}; {scores[i]} | {'{:.4f}'.format(distance[i][0]) }}}"
        })
    res_str = ""
    for s in get_dist(list(map(lambda x: x["indices"], data_collected))):
        local_l = data_collected[s]
        local_list = list(zip(local_l["indices"], local_l["value"]))
        res = f"{format_list(local_l["trace"].tolist())} -> "
        for index, value in sorted(local_list, key=lambda x: x[1], reverse=True):
            res += f"[{f'{index:2}'}]: {'{:.4f}'.format(value)} "
        res_str += res + f"{" "*(150 - len(res))} {local_l["neighbour"]}\n"
    return res_str

def show_clusters(x, y):
    vectors, mask = interpreter.attended_context(to_one_hot(x), y.reshape(-1, 1))
    indices_y = group_by(y[mask].reshape(-1, 1).cpu().numpy(), lambda e: e.data.tobytes(),)
    with open(f"results/cluster/alpha={ALPHA}, epsilon={EPSILON}.txt", "w") as file:
        for event, context_mask in indices_y:
            event = ord(event.decode('ascii')[0])   
            if event not in interpreter.tree:
                file.write(f"{"<"*50}[{event}]{">"*50}\n")
                continue
            file.write(f"{"="*50}[{event}]{"="*50}\n")
            vectors_, inverse, _ = sp_unique(vectors[context_mask])
            distance, neighbours = interpreter.tree[event].query(
                X               = vectors_.toarray(),
                return_distance = True,
                dualtree        = vectors_.shape[0] >= 1e3, # Optimization
            )
            neighbours = interpreter.tree[event].get_arrays()[1][neighbours][:, 0]
            scores = np.asarray([interpreter.labels[event][neighbour] for neighbour in neighbours])
            file.write(format_attention(x[mask], context_mask, vectors, neighbours[inverse], distance[inverse], scores[inverse]) + "\n")

def interpret(context_passed, events_passed, threshold=0.2):
    c = to_one_hot(context_passed)
    e = events_passed.reshape(-1, 1)
    interpreter.threshold = threshold
    if not attention_query:
        return interpreter.predict(X=c, y=e, iterations=0)
    return interpreter.predict(X=c, y=e)

def bim_no_iter(context_chosen, event_chosen, iters):
    context_processed = to_cuda(to_one_hot(context_chosen).clone().detach())
    original_context = to_cuda(to_one_hot(context_chosen).clone().detach())
    context_processed.requires_grad_(True)
    output = builder.predict(context_processed)
    loss = criterion(output[0][0], event_chosen)
    context_processed.retain_grad()
    loss.backward(retain_graph=True)
    context_processed = get_perturbation_change(context_processed + (ALPHA * context_processed.grad.sign()) * iters, original_context)
    return to_cuda(torch.tensor(torch.argmax(context_processed, dim=-1).tolist()[0]))

def run_bim(context_to_process, events_to_process):
    perturbed_collected_main = []
    perturbed_indices_main = []
    states = [0, 0, 0]
    length = len(context_to_process)
    iters = tqdm(range(length))
    iters_to_flip = get_iter_count(context_to_process, events_to_process)
    for current_trace_num in iters:
        con, e = context_to_process[current_trace_num], events_to_process.unsqueeze(1)[current_trace_num]
        con.resize_(1, con.size()[-1])
        if get_performance(con, e[0]):
            states[0] += 1
            continue
        perturbed = bim_no_iter(con, e, iters=iters_to_flip)
        perturbed.resize_(1, perturbed.size()[-1])
        if get_performance(perturbed, e[0]):
            perturbed_collected_main.append(perturbed.tolist()[0])
            perturbed_indices_main.append(current_trace_num)
            states[1] += 1
        else:
            states[2] += 1
    print(f"incorrect={states[0]} perturbed={states[1]} timeout={states[2]}")
    return to_cuda(torch.tensor(perturbed_collected_main)), to_cuda(torch.tensor(perturbed_indices_main))

In [221]:
perturbed_collected, perturbed_indices, perturb_distribution = get_perturbations_or_file(context_test, events_test)

Loading perturbed/collected/ALPHA=0.01, EPSILON=0.5.pt
Loading perturbed/indices/ALPHA=0.01, EPSILON=0.5.pt
Loading perturbed/distribution/ALPHA=0.01, EPSILON=0.5.pt


In [222]:
pd.Series(labels_test.cpu()).value_counts().sort_index()

1      65
2     454
3    3864
5       9
Name: count, dtype: int64

In [223]:
res_normal = interpret(context_test, events_test)
pd.Series(res_normal).value_counts().sort_index()

-3.0     467
-1.0     490
 2.0     378
 3.0    3025
 5.0      32
Name: count, dtype: int64

In [224]:
print("Unique:", len(get_unique_indices_per_row(perturbed_collected)))
res_perturbed_only = interpret(perturbed_collected, events_test[perturbed_indices])
pd.Series(res_perturbed_only).value_counts().sort_index()

Unique: 2386


-3.0     118
-1.0    2745
 2.0      16
 3.0     104
Name: count, dtype: int64

In [225]:
context_test_original_perturbed = context_test.clone().detach()
context_test_original_perturbed[perturbed_indices] = perturbed_collected
c_indices_perturbed = get_unique_indices_per_row(context_test_original_perturbed)
res_combined_perturbed = interpret(context_test_original_perturbed[c_indices_perturbed], events_test[c_indices_perturbed])
pd.Series(res_combined_perturbed).value_counts().sort_index()

-3.0     356
-1.0    2640
 2.0     110
 3.0     660
 5.0      14
Name: count, dtype: int64

In [249]:
perturbed_minimized = get_shortcuts_or_file(
    perturbed_collected, 
    perturbed_indices,
    context_test, 
    events_test
)

100%|██████████| 2983/2983 [00:44<00:00, 66.65it/s]


In [250]:
print("Unique:", len(get_unique_indices_per_row(perturbed_minimized)))
res_perturbed_shortcuts = interpret(perturbed_minimized, events_test[perturbed_indices])
pd.Series(res_perturbed_shortcuts).value_counts().sort_index()

Unique: 2960


-3.0    1398
-1.0     393
 2.0     153
 3.0    1033
 5.0       6
Name: count, dtype: int64

In [251]:
context_test_original_perturbed_shortcuts = context_test.clone().detach()
context_test_original_perturbed_shortcuts[perturbed_indices] = perturbed_minimized
c_indices_perturbed_shortcuts = get_unique_indices_per_row(context_test_original_perturbed_shortcuts)
res_combined_perturbed_shortcuts = interpret(context_test_original_perturbed_shortcuts[c_indices_perturbed_shortcuts], events_test[c_indices_perturbed_shortcuts])
pd.Series(res_combined_perturbed_shortcuts).value_counts().sort_index()

-3.0    1638
-1.0     873
 2.0     251
 3.0    1581
 5.0      20
Name: count, dtype: int64

In [252]:
perturbed_minimized_attention_query = get_shortcuts_attention_query_or_file(
    perturbed_collected, 
    perturbed_indices,
    context_test, 
    events_test
)

Loading perturbed/minimized_attention_query/alpha=0.01, epsilon=0.5.pt


In [255]:
print("Unique:", len(get_unique_indices_per_row(perturbed_minimized_attention_query)))
res_perturbed_shortcuts = interpret(perturbed_minimized_attention_query, events_test[perturbed_indices])
pd.Series(res_perturbed_shortcuts).value_counts().sort_index()

Unique: 2780


-3.0     118
-1.0    2745
 2.0      16
 3.0     104
Name: count, dtype: int64

In [254]:
context_test_original_perturbed_shortcuts_attention_query = context_test.clone().detach()
context_test_original_perturbed_shortcuts_attention_query[perturbed_indices] = perturbed_minimized_attention_query
c_indices_perturbed_shortcuts_attention_query = get_unique_indices_per_row(context_test_original_perturbed_shortcuts_attention_query)
res_combined_perturbed_shortcuts_attention_query = interpret(context_test_original_perturbed_shortcuts_attention_query[c_indices_perturbed_shortcuts_attention_query], events_test[c_indices_perturbed_shortcuts_attention_query])
pd.Series(res_combined_perturbed_shortcuts_attention_query).value_counts().sort_index()

-3.0     356
-1.0    3032
 2.0     109
 3.0     661
 5.0      14
Name: count, dtype: int64

In [ ]:
show_clusters(context_test, events_test)